In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Rutas base (el notebook vive en notebooks/, los logs en ../logs)
LOGS_DIR = (Path.cwd().parent / "logs").resolve()

print(f"Librerías importadas. Usando logs en: {LOGS_DIR}")

ModuleNotFoundError: No module named 'seaborn'

In [1]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Seleccionar el log más reciente de phase3 CNP y construir Gantt reutilizable

def pick_latest(pattern: str, base: Path = LOGS_DIR) -> Path:
    files = sorted(base.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    if not files:
        raise FileNotFoundError(f"No se encontraron archivos para patrón {pattern} en {base}")
    return files[0]

cnp_file = pick_latest('simulation_phase3_cnp_*.csv')
print(f"Usando log de eventos: {cnp_file.name}")

raw_events = pd.read_csv(cnp_file)
df_starts = raw_events[raw_events['event_type'] == 'start'].copy()
df_starts['op_id'] = df_starts.groupby('job_id').cumcount()
df_starts['start_time'] = df_starts['time']
df_starts['end_time'] = df_starts['time'] + df_starts['duration']
df_starts['label'] = df_starts.apply(lambda r: f"J{int(r['job_id'])}-Op{int(r['op_id'])}", axis=1)

n_jobs_fig = 6
df_gantt = df_starts[df_starts['job_id'] < n_jobs_fig]

fig = px.timeline(
    df_gantt,
    x_start='start_time',
    x_end='end_time',
    y='machine_id',
    color='job_id',
    text='label',
    title=f'Figura V-1: Gantt phase3 CNP (primeros {n_jobs_fig} jobs)',
    labels={'machine_id': 'Máquina', 'start_time': 'Inicio', 'end_time': 'Fin', 'job_id': 'Job'}
)
fig.update_yaxes(categoryorder='category ascending')
fig.update_traces(textposition='inside', textfont_size=9)
fig.update_layout(height=500, showlegend=True, legend_title_text='Job', font=dict(size=11))
fig.show()

overlaps = 0
for m, ops in df_gantt.sort_values('start_time').groupby('machine_id'):
    ops = ops.sort_values('start_time')
    for i in range(len(ops) - 1):
        if ops.iloc[i]['end_time'] > ops.iloc[i+1]['start_time']:
            overlaps += 1
print(f"Solapamientos detectados en {n_jobs_fig} jobs: {overlaps} {'✅' if overlaps == 0 else '❌'}")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 8. Diagrama de Gantt FT06 (Gráfico V-1)

Visualización del schedule de FT06 usando el log de operaciones ejecutadas. Se evidencia no solapamiento y precedencias.

In [5]:
# Ejemplo de histograma de latencias (ajusta a tu log real)
# df_rtt = pd.read_csv('../twin_scheduler/data/latencias_rtt.csv')
# plt.figure(figsize=(8,4))
# plt.hist(df_rtt['rtt_ms'], bins=30, color='skyblue', edgecolor='k')
# plt.xlabel('RTT (ms)')
# plt.ylabel('Frecuencia')
# plt.title('Histograma de latencias RTT')
# plt.show()

# Descomenta y ajusta según tu log real

## 7. Histograma de latencias (RTT)

Si tienes logs de latencias de comunicación, visualiza la distribución.

In [2]:
# Ejemplo de curva WIP vs tiempo (ajusta a tu log real)
# df_wip = pd.read_csv('../twin_scheduler/data/wip_vs_time.csv')
# plt.figure(figsize=(10,5))
# plt.plot(df_wip['time'], df_wip['wip'], label='WIP')
# plt.xlabel('Tiempo')
# plt.ylabel('WIP')
# plt.title('Evolución del WIP en el tiempo')
# plt.legend()
# plt.grid(True)
# plt.show()

# Descomenta y ajusta según tu log real

## 6. Curva WIP vs tiempo

Visualiza la evolución del WIP (Work In Process) a lo largo del tiempo (requiere columna de tiempo y WIP en tu log).

In [ ]:
# Diagrama de Gantt de un job específico (reutiliza df_starts ya calculado)
job_id = 0  # Cambia este valor para otra instancia

df_job = df_starts[df_starts['job_id'] == job_id]
fig = px.timeline(
    df_job,
    x_start='start_time',
    x_end='end_time',
    y='machine_id',
    color='op_id',
    text='label',
    title=f'Diagrama de Gantt - Job {job_id}',
    labels={'start_time': 'Inicio', 'end_time': 'Fin', 'machine_id': 'Máquina'}
)
fig.update_yaxes(categoryorder='category ascending')
fig.update_layout(height=350)
fig.show()

## 5. Diagrama de Gantt de un job representativo

Visualiza la secuencia de operaciones de un job en el tiempo (ajusta el path y columnas según tu log de scheduling).

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='Política', y='completion_time', data=df_jobs_all, palette='Set2')
plt.title('Boxplot de tiempos de finalización por política')
plt.show()

plt.figure(figsize=(10,5))
sns.boxplot(x='Política', y='tardiness', data=df_jobs_all, palette='Set2')
plt.title('Boxplot de tardanza por política')
plt.show()

NameError: name 'df_spt' is not defined

## 4. Boxplots de makespan y tardanza

Visualiza la dispersión de los resultados entre corridas para cada política.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

sns.barplot(x='Política', y='Makespan total', data=df_resumen, ax=axes[0,0], palette='Set2')
axes[0,0].set_title('Makespan total')

sns.barplot(x='Política', y='Tardanza prom', data=df_resumen, ax=axes[0,1], palette='Set2')
axes[0,1].set_title('Tardanza promedio')

sns.barplot(x='Política', y='% tardíos', data=df_resumen, ax=axes[1,0], palette='Set2')
axes[1,0].set_title('% Jobs tardíos')

sns.barplot(x='Política', y='Tardanza std', data=df_resumen, ax=axes[1,1], palette='Set2')
axes[1,1].set_title('Desvío estándar tardanza')

plt.tight_layout()
plt.show()

## 3. Gráficos de barras comparativos

Comparación visual de makespan, tardanza promedio, % tardíos y utilización por política.

In [ ]:
# Resumen por política usando las columnas disponibles en los logs de jobs

def resumen_metricas(df: pd.DataFrame, nombre: str) -> pd.Series:
    return pd.Series({
        'Política': nombre,
        'Makespan total': df['completion_time'].max(),
        'Tardanza prom': df['tardiness'].mean(),
        'Tardanza std': df['tardiness'].std(),
        '% tardíos': (df['tardiness'] > 0).mean() * 100,
        'Jobs': len(df),
    })

resumenes = [resumen_metricas(df, pol) for pol, df in policy_dfs.items()]
df_resumen = pd.DataFrame(resumenes)
df_resumen

## 2. Tabla comparativa de desempeño

Mostramos una tabla resumen con los principales indicadores por política.

In [ ]:
# Cargar resultados de políticas desde los logs ya generados

policy_patterns = {
    'SPT': 'simulation_phase1_SPT_jobs_*.csv',
    'EDD': 'simulation_phase1_EDD_jobs_*.csv',
    'LPT': 'simulation_phase1_LPT_jobs_*.csv',
}

policy_dfs = {}
for policy, pattern in policy_patterns.items():
    file_path = pick_latest(pattern)
    df = pd.read_csv(file_path)
    df['Política'] = policy
    policy_dfs[policy] = df
    print(f"{policy}: {file_path.name} ({len(df)} filas)")

# Un solo dataframe largo para boxplots/combos
df_jobs_all = pd.concat(policy_dfs.values(), ignore_index=True)

## 1. Importar librerías y cargar datos

Ajusta la ruta de tus archivos CSV según corresponda.

# Visualización de Resultados de Scheduling

Este notebook es una plantilla para graficar y analizar los resultados experimentales de tu tesis. Incluye:
- Tablas comparativas
- Gráficos de barras
- Boxplots
- Gantt
- Curvas WIP/tiempo
- Histograma de latencias

> **Modifica las rutas y nombres de archivo según tus datos reales.**